<h3>Librerías, paquetes y funciones usadas en el cuaderno</h3>

In [1]:
import cv2
import numpy as np

<h3>Función para la obtención del color del parqué y de la zona</h3>

In [2]:
# Función para manejar los clics del ratón
def get_color(event, x, y, flags, param):
    global selected_colors
    if event == cv2.EVENT_LBUTTONDOWN:
        selected_color = frame[y, x]  # Obtener el color en el punto clicado
        selected_colors.append(selected_color)
        print(f"Color seleccionado (BGR): {selected_color}")
        if len(selected_colors) == 2:  # Si ya se seleccionaron dos colores, cerrar la ventana
            cv2.destroyWindow("Parquet y zona.")

<h3>Selección de colores del parqué y de la zona para generar una máscara</h3>

In [3]:
# Leer el video
video_path = "./assets/videos/dal-lac1.mp4"
cap = cv2.VideoCapture(video_path)

# Variables globales para almacenar los colores seleccionados
selected_colors = []

ret, frame = cap.read()  # Leer el primer fotograma
if not ret:
    print("Error al leer el video.")
    cap.release()
    exit()

# Mostrar el primer fotograma para seleccionar colores
cv2.imshow("Parquet y zona.", frame)
cv2.setMouseCallback("Parquet y zona.", get_color)

print("1. Haz clic en el parquet para seleccionar un color.")
print("2. Haz clic en la zona bajo la canasta para seleccionar otro color.")


# Esperar hasta que el usuario seleccione dos colores
while len(selected_colors) < 2:
    cv2.waitKey(1)

# Convertir los colores seleccionados a espacio HSV
selected_colors_hsv = [cv2.cvtColor(np.uint8([[color]]), cv2.COLOR_BGR2HSV)[0][0] for color in selected_colors]

# Crear máscaras para cada color seleccionado
masks = []
for color_hsv in selected_colors_hsv:
    lower_bound_hsv = np.clip(color_hsv - [10, 50, 50], 0, 255)
    upper_bound_hsv = np.clip(color_hsv + [10, 50, 50], 0, 255)

    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gaussian_hsv = cv2.GaussianBlur(frame_hsv, (5,5), 0)
    #gaussian_hsv = cv2.bilateralFilter(frame_hsv, 6, 30, 30)

    mask = cv2.inRange(gaussian_hsv, lower_bound_hsv, upper_bound_hsv)
    masks.append(mask)

# Combinar las máscaras
combined_mask = cv2.bitwise_or(masks[0], masks[1])

# Invertir la máscara combinada y aplicar para eliminar las áreas seleccionadas
mask_inv = cv2.bitwise_not(combined_mask)
masked_frame = cv2.bitwise_and(frame, frame, mask=mask_inv)

# Mostrar los resultados
cv2.imshow("Original", frame)
cv2.imshow("Parquet y zona eliminadas", masked_frame)

cv2.waitKey(0)
cv2.destroyAllWindows()
cap.release()


1. Haz clic en el parquet para seleccionar un color.
2. Haz clic en la zona bajo la canasta para seleccionar otro color.
Color seleccionado (BGR): [161 198 236]
Color seleccionado (BGR): [ 83  50 202]


In [ ]:
import numpy as np
import cv2
from skimage import transform

# Captura cuatro clics del ratón (parte uno)
def get_points(event, x, y, flags, param):
    puntos = param["points"]
    img = param["image"].copy()

    if event == cv2.EVENT_LBUTTONDOWN:  # Botón izquierdo
        puntos.append((x, y))  
        
        # Dibujar en la imagen
        cv2.circle(img, (x, y), 5, (0, 255, 0), -1)
        cv2.imshow(param["wname"], img)

        # Cuarto puntos, condición de parada
        if len(puntos) == 4:
            cv2.destroyAllWindows()
            print("Coordenadas imagen:", puntos)

# Movimiento del puntero (parte 2)
def mouse_event(event, x, y, flags, param):
    homo = param["tform"]
    img = param["image"]

    if event == cv2.EVENT_MOUSEMOVE:  
        # Transformar posición del ratón a imagen destino
        print("uwu")
        punto_A = np.array([[x, y]])  # Puntero 
        punto_B = homo(punto_A)  # Transformar a imagen destino

        # Dibuja en la imagen destino
        print("wuwu")
        imgtmp = img.copy()
        cv2.circle(imgtmp, (int(punto_B[0][0]), int(punto_B[0][1])), 5, (0, 255, 0), -1)
        cv2.imshow(param["wname"], imgtmp)

############## 1. RECOPILA LOS CUATRO PUNTOS EN AMBAS IMÁGENES
#Inicializa listas depuntos
puntosA = []
puntosB = []

video_path = "./assets/videos/okc-nyk.mp4"
cap = cv2.VideoCapture(video_path)

# Variables globales para almacenar los colores seleccionados
selected_colors = []

ret, frame = cap.read()  # Leer el primer fotograma
if not ret:
    print("Error al leer el video.")
    cap.release()
    exit()

#Lee imágenes, crea copias de trabajo
diagrama = cv2.imread('./assets/court_diagrams/black_court.png')

frametmp = frame.copy()

# aqui toco cosas
frametmp = masked_frame

diagramatmp = diagrama.copy()

#Vista del campo
cv2.imshow("Vista", frametmp) 
params = {
    "points": puntosA,
    "image": frametmp, 
    "wname": "Vista"
}
cv2.setMouseCallback("Vista", get_points, params)

# Selecciona cuatro puntos o cierra ventana
cv2.waitKey(0)

#Vista ceintal
cv2.imshow("Mapa", diagrama) 
params = {
    "points": puntosB,
    "image": diagramatmp, 
    "wname": "Mapa"
}
cv2.setMouseCallback("Mapa", get_points, params)

# Selecciona cuatro puntos o cierra ventana
cv2.waitKey(0)
cv2.destroyAllWindows()

print("hola")
print("Coordenadas campo:", puntosA)
print("Coordenadas diagrama:", puntosB)
#Transformación de los puntos https://scikit-image.org/docs/stable/auto_examples/transform/plot_transform_types.html
tform = transform.estimate_transform('projective', np.array(puntosA), np.array(puntosB))
print("adios")

###### 2. MUESTRA POSICIÓN DEL RATÓN EN EL CAMPO, EN LA IMAGEN DEL MAPA
# Mostrar las dos imágenes
cv2.imshow("Vista", frame)
cv2.imshow("Mapa", diagrama)

params = {
    "tform": tform,
    "image": diagrama, 
    "wname": "Mapa"
}

# Evento de movimiento del ratón
cv2.setMouseCallback("Vista", mouse_event, params)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
import cv2
import numpy as np
from skimage import transform
from ultralytics import YOLO  #Necesario para detectar con YOLO

#Intersección enter dos líneas
def line_intersection(line1, line2):
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2

    # Calcular  pendientes e intersecciones
    denom = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
    if denom == 0:  # Líneas paralelas o coincidentes
        return None

    intersect_x = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denom
    intersect_y = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denom

    return (intersect_x, intersect_y)

#Comprueba que el punto esté dentro de la imagen
def is_within_image(point, image_width, image_height):
    x, y = point
    return 0 <= x < image_width and 0 <= y < image_height

#Carga modelo YOLO
detect_model = YOLO('./yolo_models/yolo11n.pt') #Contenedores
segment_model = YOLO("./yolo_models/segment.pt")
# Etiqueta de las clases o clase de interés
classNames = ["person"]

# Carga imagen
image = masked_frame
height, width, _ = image.shape  
# Convierte a HSV
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
# Muestra entrada
cv2.imshow("Input", image)

# Rango de verde de interés en HSV
# Se selecciónan los valores minimos y máximos con los del parquet y zona

# Máscara de zona verde
mask = mask_inv
# Muestra máscara
cv2.imshow("Máscara", mask)

masked_frame_gray = cv2.cvtColor(masked_frame, cv2.COLOR_BGR2GRAY)
canny_masked_frame = cv2.Canny(masked_frame_gray, 50, 150)

cv2.imshow("canny", canny_masked_frame)

# Convierte a grises y detecta de bordes
# gray = cv2.cvtColor(canny_masked_frame, cv2.COLOR_BGR2GRAY)

# Detecta líneas con la transformada de Hough
min_line_length = canny_masked_frame.shape[0] // 5  # Longitud mínima de la linea
lineas_campo = cv2.HoughLinesP(canny_masked_frame, 1, np.pi / 180, threshold=30, minLineLength=min_line_length, maxLineGap=5)

#cv2.imshow("masked_image", masked_image)

# Filtrar las nlineas líneas más próximas a la parte inferior de la imagen
nlineas = 10
if lineas_campo is not None:
    lineas_campo = sorted(lineas_campo, key=lambda l: min(l[0][1], l[0][3]), reverse=True)
    lineas_campo = lineas_campo[:10]

# Inicializa grupos de líneas y umbral de orientación para separar
group_1 = []  
group_2 = []  
angle_threshold = 10

# Agrupa líneas por orientación
if lineas_campo is not None:
    for line in lineas_campo:
        x1, y1, x2, y2 = line[0]
        
        # Evita división por cero en líneas verticales
        if x2 - x1 == 0:
            angle = 90  
        else:
            # Ángulo en grados
            angle = np.degrees(np.arctan((y2 - y1) / (x2 - x1)))
            angle = abs(angle) 
        
        # Asignar grupo
        if abs(angle) < angle_threshold:  # Cercano a horizontal
            group_1.append(line)
        else:  # Resto
            group_2.append(line)

# Dibuja cada grupo con un color
if group_1:
    for line in group_1:
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Azul para grupo 1

if group_2:
    for line in group_2:
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Verde para grupo 2

#INTERSECCIONES
# Obtener las intersecciones entre líneas de grupo diferente
image_height, image_width = image.shape[:2]
intersections = []

for i in range(len(group_1)):
    for j in range(len(group_2)):
        line1 = group_1[i][0]
        line2 = group_2[j][0]

        intersection = line_intersection(line1, line2)
        # Chequea que esté en la imagen
        if intersection and is_within_image(intersection, image_width, image_height):
            intersections.append(intersection)

# Dibuja  intersecciones en la imagen
for point in intersections:
    x, y = point
    cv2.circle(image, (int(x), int(y)), 5, (255, 0, 255), -1)

# Muestra imagen con líneas e intersecciones
cv2.imshow("Líneas y esquinas detectadas", image)


cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'masked_frame' is not defined

Segmentación

In [26]:
import cv2
import numpy as np
from ultralytics import YOLO  #Necesario para detectar con YOLO

#Intersección enter dos líneas
def line_intersection(line1, line2):
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2

    # Calcular  pendientes e intersecciones
    denom = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
    if denom == 0:  # Líneas paralelas o coincidentes
        return None

    intersect_x = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denom
    intersect_y = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denom

    return (intersect_x, intersect_y)

#Comprueba que el punto esté dentro de la imagen
def is_within_image(point, image_width, image_height):
    x, y = point
    return 0 <= x < image_width and 0 <= y < image_height


# Cargar el modelo YOLO entrenado
model = YOLO("./yolo_models/segment.pt")

video_path = "./assets/videos/okc-nyk.mp4"
# Cargar el video de entrada
cap = cv2.VideoCapture(video_path)

# Obtener detalles del video
fourcc = cv2.VideoWriter_fourcc(*"avc1")  # Codec para el video de salida
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Procesar cada frame del video
ret, frame = cap.read()
if not ret:
    print("Error al leer el video.")
    cap.release()
    exit()

# Realizar detección en el frame
results = model.predict(frame, task='segment')
# Dimensiones del frame original
frame_height, frame_width = frame.shape[:2]

# Se preestablecen las coordenadas del segmento a investigar
xmin, ymin, xmax, ymax = 0,0,0,0
cropped_zone = 0
# Iterar sobre los resultados
for result in results:
    masks = result.masks.data.cpu().numpy()  # Máscaras (NumPy array)
    classes = result.boxes.cls.cpu().numpy()  # Clases detectadas
    boxes = result.boxes.xyxy.cpu().numpy()  # Bounding boxes
    image = frame.copy()  # Crear copia del frame original para visualización

    # Aplicar y mostrar máscaras
    for i, cls in enumerate(classes):
        if model.names[cls] == 'three_second_area':
            # Redimensionar la máscara al tamaño del frame original
            zone_mask = (masks[i] * 255).astype(np.uint8)  # Escalar la máscara (0-255)
            resized_zone_mask = cv2.resize(zone_mask, (frame_width, frame_height), interpolation=cv2.INTER_NEAREST)

            # Invertir la máscara
            zone_mask_inv = cv2.bitwise_not(resized_zone_mask)

            # Aplicar la máscara al frame original
            combined = cv2.bitwise_and(image, image, mask=zone_mask_inv)
            
            xmin, ymin, xmax, ymax = boxes[i].astype(int)
            cropped_zone = combined[ymin:ymax, xmin:xmax]

# Mostrar la zona en pantalla
cv2.imshow("La zona", cropped_zone)

# Detección de líneas en la zona designada
image = cropped_zone.copy()
gaussian_image = cv2.GaussianBlur(image, (17,17), 0)



# Forma alternativa
masked_frame_gray = cv2.cvtColor(gaussian_image, cv2.COLOR_BGR2GRAY)

# Forma utilizada
mask = cv2.inRange(gaussian_image, 0,1)
cv2.imshow("mascara", mask)
gaussian_masked_frame = cv2.GaussianBlur(mask, (17,17), 0)
cv2.imshow("gaussian mask", gaussian_masked_frame)
canny_masked_frame = cv2.Canny(gaussian_masked_frame, 30, 150)
cv2.imshow("canny", canny_masked_frame)
gaussian_canny_masked_frame = cv2.bilateralFilter(canny_masked_frame, 6, 75, 75)

# Detecta líneas con la transformada de Hough
min_line_length = canny_masked_frame.shape[0]*0.5  # Longitud mínima de la linea
lineas_campo = cv2.HoughLinesP(gaussian_canny_masked_frame, rho=1, theta=np.pi / 180, threshold=15, minLineLength=min_line_length, maxLineGap=105)

# Filtrar las nlineas líneas más próximas a la parte inferior de la imagen
nlineas = 10
if lineas_campo is not None:
    lineas_campo = sorted(lineas_campo, key=lambda l: min(l[0][1], l[0][3]), reverse=True)
    lineas_campo = lineas_campo[:10]

# Inicializa grupos de líneas y umbral de orientación para separar
group_1 = []  
group_2 = []  
angle_threshold = 20

# Agrupa líneas por orientación
if lineas_campo is not None:
    for line in lineas_campo:
        print(line)
        x1, y1, x2, y2 = line[0]
        
        # Evita división por cero en líneas verticales
        if x2 - x1 == 0:
            angle = 90  
        else:
            # Ángulo en grados
            angle = np.degrees(np.arctan((y2 - y1) / (x2 - x1)))
            angle = abs(angle) 
        
        # Asignar grupo
        if abs(angle) < angle_threshold:  # Cercano a horizontal
            group_1.append(line)
        else:  # Resto
            group_2.append(line)

# Dibuja cada grupo con un color
if group_1:
    for line in group_1:
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Azul para grupo 1

if group_2:
    for line in group_2:
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Verde para grupo 2

#INTERSECCIONES
# Obtener las intersecciones entre líneas de grupo diferente
image_height, image_width = image.shape[:2]
intersections = []

for i in range(len(group_1)):
    for j in range(len(group_2)):
        line1 = group_1[i][0]
        line2 = group_2[j][0]

        intersection = line_intersection(line1, line2)
        # Chequea que esté en la imagen
        if intersection:
            intersections.append(intersection)
        # Dibujar el punto    
        x,y = intersection
        if is_within_image(intersection, image_width, image_height):
            cv2.circle(image, (int(x), int(y)), 5, (255, 0, 255), -1)
        

# "intersections" contiene todos los puntos de intersección de las líneas encontradas en el espacio
print("Puntos de intersección: ")
print(intersections)


# Muestra imagen con líneas e intersecciones
cv2.imshow("Líneas y esquinas detectadas", image)

# Esperar a que se presione una tecla y cerrar ventana
cv2.waitKey(0)

cv2.destroyAllWindows()


0: 384x640 1 backboard, 1 backboard_inner_square, 1 court, 1 hoop, 1 net, 1 three_second_area, 1 two_point_area, 30.7ms
Speed: 7.0ms preprocess, 30.7ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
[[116 139 332 124]]
[[135 143 515  97]]
[[236 132 577  91]]
[[  9  63 120 141]]
[[  9  62 123 142]]
[[  9  53 332  19]]
[[ 11  51 340  17]]
[[ 24  49 344  15]]
[[220  27 346  14]]
[[417  14 575  81]]
Puntos de intersección: 
[(117.05007292173067, 138.92707826932426), (118.47962085308058, 138.82780410742495), (627.9387246170288, 103.448699679373), (124.63218787774132, 144.2550509411155), (125.98081023454158, 144.09179665581865), (591.0278660474212, 87.79662674162796), (126.01165366464413, 145.22440527785804), (127.36320961381986, 145.06190148338237), (593.8145268542199, 88.97831202046036), (-3.3767605633802815, 54.30281690140845), (-2.152173913043478, 54.17391304347826), (409.5424795468848, 10.837633731906859), (-5.631064003261312, 52.718711781492054), (-4.406212743409723,

In [27]:
frametmp = frame.copy()
# Filtrar puntos muy cercanos
#filtered_intersections = []
#for i in range(len(intersections)):
#    if len(filtered_intersections) == 0:
#        filtered_intersections.append(intersections[i])
#        continue
#    valid = True
#    for j in range(len(filtered_intersections)):
#        if np.linalg.norm(np.array(intersections[i]) - np.array(filtered_intersections[j])) < 50:
#            valid = False
#            break
#
#    if valid:
#        filtered_intersections.append(intersections[i])
#
#print("Puntos de intersección: ")
#print(intersections)
#print("Puntos de intersección filtrados: ")
#print(filtered_intersections)

# Desnormalizar las coordenadas de las intersecciones
#for point in filtered_intersections:
#    x = int(point[0] + xmin)
#    y = int(point[1] + ymin)
#    cv2.circle(frametmp, (x, y), 5, (255, 0, 0), -1)
#    cv2.putText(frametmp, f"({x}, {y})", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
#    # Sustituir point por (x, y) para desnormalizar
#    filtered_intersections[filtered_intersections.index(point)] = (x, y)

denormalized_intersections = []
for point in intersections:
    x = int(point[0] + xmin)
    y = int(point[1] + ymin)
    denormalized_intersections.append((x, y))
    cv2.circle(frametmp, (x, y), 5, (255, 0, 0), -1)
    cv2.putText(frametmp, f"({x}, {y})", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)


# Puntos de la imagen para la homografía
print("Puntos de intersección desnormalizados: ")
print(denormalized_intersections)

cv2.imshow("Líneas y esquinas detectadas", frametmp)
# Esperar a que se presione una tecla y cerrar ventana
cv2.waitKey(0)


cv2.destroyAllWindows()

Puntos de intersección desnormalizados: 
[(575, 448), (576, 448), (1085, 413), (582, 454), (583, 454), (1049, 397), (584, 455), (585, 455), (1051, 398), (454, 364), (455, 364), (867, 320), (452, 362), (453, 362), (865, 320), (451, 362), (452, 362), (862, 318), (449, 360), (450, 360), (861, 318)]


In [29]:
def mouse_event(event, x, y, flags, param):
    homo = param["tform"]
    img = param["image"]

    if event == cv2.EVENT_MOUSEMOVE:  
        # Transformar posición del ratón a imagen destino
        punto_A = np.array([[x, y]])  # Puntero
        punto_B = homo(punto_A)  # Transformar a imagen destino

        # Dibuja en la imagen destino
        imgtmp = img.copy()
        cv2.circle(imgtmp, (int(punto_B[0][0]), int(punto_B[0][1])), 5, (0, 255, 0), -1)
        cv2.imshow(param["wname"], imgtmp)


def obtener_puntos_cancha_ordenados(denormalized_intersections):
    puntos_ordenados = []

    # Detectar en qué lado de la cancha está el aro
    # Si la intersección cuya x es mayor está a la derecha, entonces el aro está a la derecha
    furthest_point = max(denormalized_intersections, key=lambda p: p[0])
    lado_derecho = furthest_point[0] > frame_width // 2

    # Punto A
    if lado_derecho:
        # Punto A (xmin, y_A)
        closest_point = min(denormalized_intersections, key=lambda p: p[0])
        print("Punto A: ", closest_point)
        puntos_ordenados.append(closest_point)
    else:
        # Punto A (xmax, y_A)
        print("Punto A: ", furthest_point)
        puntos_ordenados.append(furthest_point)

    # Estos puntos son independientes del lado de la cancha
    # Punto B (x_B, ymax)
    lowest_point = max(denormalized_intersections, key=lambda p: p[1])
    print("Punto B: ", lowest_point)
    puntos_ordenados.append(lowest_point)

    # Punto C (x_C, ymin)
    highest_point = min(denormalized_intersections, key=lambda p: p[1])
    print("Punto C: ", highest_point)
    puntos_ordenados.append(highest_point)

    # Punto D
    if lado_derecho:
        # Punto D (xmax, y_D)
        print("Punto D: ", furthest_point)
        puntos_ordenados.append(furthest_point)

    else:
        # Punto D (xmin, y_D)
        closest_point = min(denormalized_intersections, key=lambda p: p[0])
        print("Punto D: ", closest_point)
        puntos_ordenados.append(closest_point)

    return puntos_ordenados, lado_derecho

In [30]:
from skimage import transform


diagrama = cv2.imread('./assets/court_diagrams/white_court.png')
# Puntos del diagrama para la homografía: [A, B, C, D]
puntos_diagrama_lado_derecho = [(1509, 436), (1509, 727), (1854, 436), (1854, 727)]
puntos_diagrama_lado_izquierdo = [(490, 436), (490, 727), (145, 436), (145, 727)]



puntos_cancha, lado_derecho = obtener_puntos_cancha_ordenados(denormalized_intersections)

print("Puntos de la cancha: ")
print(puntos_cancha)

if lado_derecho:
    puntos_diagrama = puntos_diagrama_lado_derecho
else:
    puntos_diagrama = puntos_diagrama_lado_izquierdo
# Homografía
tform = transform.estimate_transform('projective', np.array(puntos_cancha), np.array(puntos_diagrama))

cv2.imshow("Partido", frame)
cv2.imshow("Diagrama", diagrama)

params = {
    "tform": tform,
    "image": diagrama, 
    "wname": "Diagrama"
}

cv2.setMouseCallback("Partido", mouse_event, params)

cv2.waitKey(0)
cv2.destroyAllWindows()
    



Punto A:  (449, 360)
Punto B:  (584, 455)
Punto C:  (862, 318)
Punto D:  (1085, 413)
Puntos de la cancha: 
[(449, 360), (584, 455), (862, 318), (1085, 413)]


DETECCIÓN DE JUGADORES Y CLASIFICACIÓN DE EQUIPOS

In [31]:
import cv2
import math
import numpy as np
from ultralytics import YOLO
from sklearn.cluster import KMeans

# Función para obtener el ROI del área central de la bbox
def get_central_roi(frame, x1, y1, x2, y2):
    # Coordenadas de la bbox
    width = x2 - x1
    height = y2 - y1

    # Definir subárea como proporción de la bbox
    sub_x1 = int(x1 + width * 0.3)  # Dejar un 30% de margen a los lados
    sub_x2 = int(x2 - width * 0.3)  # Dejar un 30% de margen a los lados
    sub_y1 = int(y1 + height * 0.3)  # Subárea empieza al 30% de la altura
    sub_y2 = int(y1 + height * 0.6)  # Subárea llega al 60% de la altura

    # Recortar y devolver el ROI
    roi = frame[sub_y1:sub_y2, sub_x1:sub_x2]
    return roi

# Función para obtener los colores de los equipos
def classifyTeam(player_roi):
    
    # Convertir a HSV y preparar los datos para KMeans (matriz de (n_pixels, 3))
    roi_hsv = cv2.cvtColor(player_roi, cv2.COLOR_BGR2HSV)
    pixels = roi_hsv.reshape(-1, 3)

    # Aplicar KMeans para obtener los colores dominantes
    k = 2
    kmeans = KMeans(n_clusters=k, random_state=0).fit(pixels)
    dominant_colors = kmeans.cluster_centers_
    labels, counts = np.unique(kmeans.labels_, return_counts=True)

    # Obtener el color más dominante
    dominant_cluster = labels[np.argmax(counts)]
    dominant_color = dominant_colors[dominant_cluster]

    # Si no se han clasificado los colores de los equipos, asignar el color a uno de ellos
    if np.array_equal(team_colors['A'], np.array([0,0,0])):
        team_colors['A'] = dominant_color
        return 'A'
    
    # Si ya se ha clasificado el color de un equipo, asignar el color al otro
    elif np.array_equal(team_colors['B'], np.array([0,0,0])):
        # Si el color es muy similar al del equipo A, evita asignarlo a B
        if np.linalg.norm(dominant_color - team_colors['A']) < 100:
            return 'A'
        team_colors['B'] = dominant_color
        return 'B'
    
    # Si ya se han clasificado ambos colores, asignar el color al equipo más cercano
    else:
        if np.linalg.norm(dominant_color - team_colors['A']) < np.linalg.norm(dominant_color - team_colors['B']):
            return 'A'
        else:
            return 'B'

# Función para obtener el color BGR de un equipo a partir de su nombre
def getTeamBGRColor(team):
    if team not in team_colors:
        return [0, 0, 0]
    bgr_color = team_colors[team].astype(np.uint8)
    bgr_color = cv2.cvtColor(bgr_color[np.newaxis, np.newaxis, :], cv2.COLOR_HSV2BGR)[0, 0]
    return bgr_color.tolist()

def drawBBox(frame, x1, y1, x2, y2, team, label):
    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), getTeamBGRColor(team), 2)
    cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 200, 100), 2)

def drawPosition(frame, position, position_label):
    cv2.circle(frame, (int(position[0]), int(position[1])), 5, (0, 0, 255), -1)
    #cv2.ellipse(frame, (int(position[0]), int(position[1])), (15, 5), 0, 0, 360, (0, 0, 255), -1)
    cv2.putText(frame, position_label, (int(position[0]) - 50, int(position[1]) + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)


# Cargar el modelo YOLO entrenado
model = YOLO("./yolo_models/bpdv1.pt")

video_path = "./assets/videos/okc-nyk.mp4"
# Cargar el video de entrada
cap = cv2.VideoCapture(video_path)

# Obtener detalles del video
fourcc = cv2.VideoWriter_fourcc(*"avc1")  # Codec para el video de salida
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

ret, frame = cap.read()
if not ret:
    print("Error al leer el video.")
    cap.release()
    exit()


# Inicializar colores de los equipos
team_colors = {'A': np.array([0,0,0]), 'B': np.array([0,0,0])}
team_A = []
team_B = []
referees = []


# Realizar detección en el frame
results = model(frame)

# Análisis de las detecciones
for result in results[0].boxes.data.tolist():  # Obtener los resultados como lista
    x1, y1, x2, y2, conf, cls = result  # Coordenadas, confianza y clase
    cls = int(cls)

    # Filtrar solo por las clases deseadas
    if model.names[cls] in ['player', 'referee'] and conf > 0.35:

        # Calcular la posición como el punto medio del borde inferior de la bbox
        position = (int((x1 + x2) / 2), int(y2))
        position_label = f"x:{int(position[0])} y:{int(position[1])}"

        label = f"{model.names[cls]} {conf:.2f}"

        # Si la clase es 'player', clasificar el equipo y dibujar la bbox en el frame
        if model.names[cls] == 'player':

            player_roi = get_central_roi(frame, int(x1), int(y1), int(x2), int(y2))

            team = classifyTeam(player_roi)

            drawBBox(frame, x1, y1, x2, y2, team, label)

            if team == 'A':
                team_A.append(position)
            else:
                team_B.append(position)

        # Si la clase es 'referee', dibujar la bbox en el frame
        if model.names[cls] == 'referee':
            drawBBox(frame, x1, y1, x2, y2, 'referee', label)
            referees.append(position)

        # Dibujar la posición en el frame
        drawPosition(frame, position, position_label)

print(f"Equipo A: {team_A}")
print(f"Equipo B: {team_B}")
print(f"Árbitros: {referees}")

cv2.imshow('Resultados', frame)
# Esperar 0ms para salir si se presiona la tecla 'q'
cv2.waitKey(0) & 0xFF == ord('q')
# Liberar recursos
cap.release()
cv2.destroyAllWindows()


0: 384x640 1 ball, 8 players, 28.5ms
Speed: 3.0ms preprocess, 28.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Equipo A: [(429, 566), (466, 376), (591, 258), (648, 307), (704, 437), (744, 405)]
Equipo B: [(340, 374), (751, 502)]
Árbitros: []


TRASLADAR JUGADORES A HOMOGRAFÍA

In [35]:
def drawDetections(diagram, team_A, team_B, referees):
    for point in team_A:
        diagram_point = tform(np.array(point))
        cv2.circle(diagram, (int(diagram_point[0][0]), int(diagram_point[0][1])), 15, getTeamBGRColor("A"), -1)

    for point in team_B:
        diagram_point = tform(point)
        cv2.circle(diagram, (int(diagram_point[0][0]), int(diagram_point[0][1])), 15, getTeamBGRColor("B"), -1)

    for point in referees:
        diagram_point = tform(point)
        cv2.circle(diagram, (int(diagram_point[0][0]), int(diagram_point[0][1])), 5, (255, 255, 255), -1)


diagramatmp = diagrama.copy()

drawDetections(diagramatmp, team_A, team_B, referees)

cv2.imshow("Diagrama", diagramatmp)
cv2.imshow("Partido", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [39]:
# Cargar el modelo YOLO entrenado
model = YOLO("./yolo_models/bpdv1.pt")

video_path = "./assets/videos/okc-nyk.mp4"
# Cargar el video de entrada
cap = cv2.VideoCapture(video_path)

# Obtener detalles del video
fourcc = cv2.VideoWriter_fourcc(*"avc1")  # Codec para el video de salida
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Configurar el video de salida
output_path = "video_resultado.mp4"
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

diagrama = cv2.imread('./assets/court_diagrams/white_court.png')

# Inicializar colores de los equipos
team_colors = {'A': np.array([0,0,0]), 'B': np.array([0,0,0])}
team_A = []
team_B = []
referees = []

# Procesar cada frame del video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    diagramatmp = diagrama.copy()

    # Realizar detección en el frame
    results = model(frame)

    # Análisis de las detecciones
    for result in results[0].boxes.data.tolist():  # Obtener los resultados como lista
        x1, y1, x2, y2, conf, cls = result  # Coordenadas, confianza y clase
        cls = int(cls)

        # Filtrar solo por las clases deseadas
        if model.names[cls] in ['player', 'referee'] and conf > 0.35:

            # Calcular la posición como el punto medio del borde inferior de la bbox
            position = (int((x1 + x2) / 2), y2)
            position_label = f"x:{int(position[0])} y:{int(position[1])}"

            label = f"{model.names[cls]} {conf:.2f}"

            # Si la clase es 'player', clasificar el equipo y dibujar la bbox en el frame
            if model.names[cls] == 'player':

                player_roi = get_central_roi(frame, int(x1), int(y1), int(x2), int(y2))

                team = classifyTeam(player_roi)

                drawBBox(frame, x1, y1, x2, y2, team, label)

                if team == 'A':
                    team_A.append(position)
                else:
                    team_B.append(position)


            # Si la clase es 'referee', dibujar la bbox en el frame
            if model.names[cls] == 'referee':
                drawBBox(frame, x1, y1, x2, y2, 'referee', label)
                referees.append(position)

            # Dibujar la posición en el frame
            drawPosition(frame, position, position_label)

    drawDetections(diagramatmp, team_A, team_B, referees)

    # Mostrar el frame procesado en pantalla
    cv2.imshow('Resultados', frame)

    cv2.imshow('Diagrama', diagramatmp)

    # Escribir el frame procesado en el video de salida
    out.write(frame)

    team_A.clear()
    team_B.clear()
    referees.clear()

    # Esperar 1ms para salir si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video procesado guardado en {output_path}")


0: 384x640 1 ball, 8 players, 15.7ms
Speed: 3.0ms preprocess, 15.7ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 8 players, 22.0ms
Speed: 3.5ms preprocess, 22.0ms inference, 7.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 9 players, 2 referees, 27.6ms
Speed: 2.5ms preprocess, 27.6ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 9 players, 2 referees, 12.8ms
Speed: 1.3ms preprocess, 12.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 9 players, 1 referee, 28.1ms
Speed: 1.2ms preprocess, 28.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 9 players, 1 referee, 11.8ms
Speed: 2.1ms preprocess, 11.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 9 players, 1 referee, 14.1ms
Speed: 0.0ms preprocess, 14.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640